In [19]:
import pickle
from phystables import BasicTable, SimpleTable, make_noisy
from phystables.constants import *
import define_tables
from define_tables import WallData, GoalData, BallData, TableData
import pygame as pg
import argparse
import numpy as np
import pandas as pd


DEF_NOISE = {
"kapv": KAPV_DEF, # VonMises precision on initial velocity angle estimation
"kapb": KAPB_DEF, # VonMises precision on the bounce angle estimation
"kapm": KAPM_DEF, # VonMises precision on the motion jitter
"perr": PERR_DEF  # Gaussian error on initial position estimation
}

def _ballhit(balllist):
    blcol = list(map(lambda b: b.col, balllist))
    print("Observed ball collisions: ", blcol)

def _goalhit(ball, goal):
    gtype = get_const(goal.ret)
    print("Ball", ball.col, "hit goal", gtype)

        
def getTable(tables_file, table_id, purpleBall, goldBall, noisy, callback=False, printTable=False):
    '''
    tables_file: File name with tables
    table_id: Table from table_files
    trial_id: Determines which simulation to run 
    noisy: T/F
    callback: T/F
    '''
    with open(tables_file, 'rb') as input:
        tablesData = pickle.load(input)
    tableData = tablesData[table_id]
    

            
    
    table = BasicTable(dims=tableData.dims)
    if printTable:
        print('\nWall Data:')
    for i, wall in enumerate(tableData.walls):
        table.add_wall(upperleft=wall.upperleft, lowerright=wall.lowerright)
        if printTable:
            print(str(i)+ ':', 'wall.upperleft', wall.upperleft)
            print(str(i)+ ':', 'wall.lowerright', wall.lowerright)

    if printTable:
            print('\nGoal Data:')
    for i, goal in enumerate(tableData.goals):
        table.add_goal(upperleft=goal.upperleft, lowerright=goal.lowerright, onreturn=goal.onreturn, color=goal.color)
        if printTable:
            print(str(i)+ ':', 'goal.upperleft', goal.upperleft)
            print(str(i)+ ':', 'goal.lowerright', goal.lowerright)
            print(str(i)+ ':', 'goal.onreturn', goal.onreturn)
            print(str(i)+ ':', 'goal.color', goal.color)
        
    balls = tableData.balls
    if purpleBall and goldBall:
        ball_id=2
    elif purpleBall:
        ball_id=0
    elif goldBall:
        ball_id=1
    if ball_id == 0 or ball_id==1:
        table.add_ball(initpos=balls[ball_id].initpos, initvel=balls[ball_id].initvel, color=balls[ball_id].color)
        if printTable:
            print('\nBall Data:')
            print(str(ball_id)+ ':'+ ':', 'balls.color', balls[ball_id].color)
            print(str(ball_id)+ ':', 'balls.initpos',    balls[ball_id].initpos)
            print(str(ball_id)+ ':', 'balls.initvel',    balls[ball_id].initvel)
    else:
        table.add_ball(initpos=balls[0].initpos, initvel=balls[0].initvel, color=balls[0].color)
        table.add_ball(initpos=balls[1].initpos, initvel=balls[1].initvel, color=balls[1].color)
        if printTable:
            print('\nBall Data:')
            print(str(0)+ ':', 'balls.color',   balls[0].color)
            print(str(0)+ ':', 'balls.initpos', balls[0].initpos)
            print(str(0)+ ':', 'balls.initvel', balls[0].initvel)
            print(str(1)+ ':', 'balls.color',   balls[1].color)
            print(str(1)+ ':', 'balls.initpos', balls[1].initpos)
            print(str(1)+ ':', 'balls.initvel', balls[1].initvel)
        
    if noisy:
        table = make_noisy(table, **DEF_NOISE)
        
    if callback:
        table.on_ballhit = _ballhit
        table.on_goalhit = _goalhit
    
    return table

def run_to_end(trial, max_time=10.):
    while True:
        r = trial.step(t=max_time, maxtime=max_time)
        if r is not None:
            if isinstance(r, list):
                r = r[0]
            return [r, trial.tm]
    
def run_simulations(tables_file, table_id, purpleBall, goldBall, noisy, num_simulations, max_time, callback=False):    
    simulations = []
    for i in range(num_simulations):
        table = getTable(tables_file, table_id, purpleBall, goldBall, noisy, callback)
        ret, simTime = run_to_end(table, max_time)
        goalHit = get_const(ret)
        ballHit = None
        ballHit = None
        simulation = [table_id, purpleBall, goldBall, goalHit, ballHit, simTime, noisy]
        simulations.append(simulation)
    return simulations

In [20]:
!python define_tables.py

pygame 2.0.0 (SDL 2.0.12, python 3.7.9)
Hello from the pygame community. https://www.pygame.org/contribute.html
Loading chipmunk for Darwin (64bit) [/Users/julio/Packages/anaconda3/envs/physenv/lib/python3.7/site-packages/pymunk/libchipmunk.dylib]


# Run Simulations and Save to Lookup Table

In [26]:
columns = ['tableID', 'purpleBall', 'goldBall', 'goalHit', 'ballHit', 'simTime', 'noisy']
LookUpTable = pd.DataFrame(columns=columns)


for i in range(1):
    simulations = run_simulations(tables_file='tables_metadata.pkl', 
                               table_id=0, 
                               purpleBall=True,
                               goldBall=True,
                               noisy=False, 
                               num_simulations=1, 
                               max_time=10)
    LookUpTable = LookUpTable.append(pd.DataFrame(data=simulations, columns=columns), ignore_index=True)  

# Save LookUpTable to file
LookUpTable.to_pickle("LookUpTable.pkl")

# Display Table Up to x columns
pd.set_option("display.max_rows", 100, "display.max_columns", 7)
LookUpTable

,tableID,purpleBall,goldBall,goalHit,ballHit,simTime,noisy
0,0,True,True,GREENGOAL,None,1.384,False


# Read File, Select Values, and Turn into List

In [12]:
# Read Table In
unpickledLookUpTable = pd.read_pickle("LookUpTable.pkl")

# Group Table and Select Group by values
simulation_groups = unpickledLookUpTable.groupby(['tableID', 'purpleBall'])
group_simulations = simulation_groups.get_group(name=(0,True)) # return dataframe for just tableID=0 and purpleBall=True
# turn into a list
group_simulations.values.tolist()

[[0, True, False, 'TIMEUP', None, 10.000999999999896, True],
 [0, True, False, 'REDGOAL', None, 1.577999999999937, True],
 [0, True, False, 'REDGOAL', None, 1.2759999999999703, True],
 [0, True, False, 'REDGOAL', None, 1.5469999999999404, True],
 [0, True, False, 'REDGOAL', None, 5.654000000000223, True],
 [0, True, False, 'REDGOAL', None, 1.6729999999999265, True],
 [0, True, False, 'REDGOAL', None, 1.5189999999999435, True],
 [0, True, False, 'REDGOAL', None, 1.6369999999999305, True],
 [0, True, False, 'REDGOAL', None, 1.567999999999938, True],
 [0, True, False, 'TIMEUP', None, 10.000999999999896, True],
 [0, True, False, 'REDGOAL', None, 3.030999999999777, True],
 [0, True, False, 'GREENGOAL', None, 1.9569999999998953, True],
 [0, True, False, 'REDGOAL', None, 1.531999999999942, True],
 [0, True, False, 'TIMEUP', None, 10.000999999999896, True],
 [0, True, False, 'GREENGOAL', None, 3.3719999999997397, True],
 [0, True, False, 'REDGOAL', None, 2.8749999999997944, True],
 [0, True, F

In [ ]:
################
# Small Oracle #
################

import random

def small_oracle(samples, num_sim_1, num_sim_2, iters=1000):
    '''
    samples:        Pre-run samples to draw from, list [Ball_1, Ball_2]
    num_sim_1:      Number of simulations for ball 1
    num_sim_2:      Number of simulations for ball 2
    samples:        Number of samples (each sample has num_sim_1 + 2 sims)
    Return tuple(# green wins, # red wins, avg total time)
    # green wins:   Number of samples where #G > #R + 0.5 * ties
    # red wins:     Number of samples where #G < #R + 0.5 * ties
    avg total time: Average total time for a sample
    '''
    g_wins = 0  # track green wins
    r_wins = 0  # track red wins
    total_time = 0  # total time

    for _ in range(iters):  # get samples
        # wins for a fixed sample
        sample_g_wins = 0
        sample_r_wins = 0
        sample_time = 0
        # simulations for ball 1
        for sim_1 in range(num_sim_1):
            table_id, outcome, time, ball = random.choice(samples[0])
            if outcome == 'GREENGOAL':
                sample_g_wins += 1
            if outcome == 'REDGOAL':
                sample_r_wins += 1
            sample_time += time
        # simulations for ball 2
        for sim_2 in range(num_sim_2):
            table_id, outcome, time, ball = random.choice(samples[1])
            if outcome == 'GREENGOAL':
                sample_g_wins += 1
            if outcome == 'REDGOAL':
                sample_r_wins += 1
            sample_time += time
        if sample_g_wins > sample_r_wins:
            g_wins += 1
        if sample_g_wins < sample_r_wins:
            r_wins += 1
        if sample_g_wins == sample_r_wins:
            g_wins += 0.5
            r_wins += 0.5
        total_time += sample_time
    return (g_wins, r_wins, total_time/iters)

In [ ]:
small_oracle([samples1, samples2], 5, 5, 10)

In [ ]:
def big_oracle(table_id, max_simulations, LookUpTable, iters):
    
    expected_utilities = []
    simulations = []
    # purple simulations
    for num_purpleBall_sims in range(max_simulations):
        for num_goldBall_sims in range(max_simulations):
            
            ground_truth = LookUpTable.groupby(['tableID', 'noisy']).get_group(name=(table_id,False)).values.tolist()
            print(ground_truth)
            
            purpleBall_sims = LookUpTable.groupby(['tableID', 'purpleBall']).get_group(name=(table_id,True)).values.tolist()            
            goldBall_sims = LookUpTable.groupby(['tableID', 'goldBall']).get_group(name=(table_id,True)).values.tolist()
            simulations = [purpleBall_sims, goldBall_sims]
            
            greeen_wins, red_wins, avg_simTime = small_oracle(simulations, 
                                                              num_purpleBall_sims, 
                                                              num_goldBall_sims, 
                                                              iters)
            
            
            
            
            
            